In [4]:
import openai
from transformers import AutoModelForCausalLM, AutoTokenizer
import argparse
import torch

### LLama 3-8B

In [ ]:
prompt = """
Goal: Memorizing the study list, where the study list consists of person-location pairs (formatted as "the person is in the location."). Please recall and list all the person-location pairs you have memorized in the study list. 

Input: The study list and question list. Study list is [  
The actor is in the airport,
The actor is in the airport,
The actor is in the airport,
The actor is in the airport,
The actor is in the airport,
The chef is in the attic,
The coach is in the bank,
The cowboy is in the barn,
The dancer is in the castle,
The detective is in the church,
The doctor is in the clinic,
The engineer is in the factory,
The farmer is in the garage,
The gardener is in the hallway,
The hippie is in the hotel,
The inventor is in the kitchen,
The judge is in the laboratory,
The maid is in the library,
The musician is in the museum,
The pilot is in the nightclub,
The pirate is in the office,
The queen is in the park,
The scientist is in the prison,
The sheriff is in the school,
The soldier is in the stadium,
The spy is in the studio,
The teacher is in the temple,
The tourist is in the theater,
The accountant is in the beach,
The artist is in the cinema,
The dentist is in the hospital,
The nurse is in the restaurant,
The writer is in the station,
The barista is in the mall,
The firefighter is in the zoo,
The lawyer is in the pharmacy
]. 
    

Output: The results of the analysis need to be in CSV (comma-separated values). 
Present the results inside a code box.  
The code box should be labeled answers_csv 
Each line of the output should contain the order and the corresponding answer.

Analysis: 
Here is a template describing how to list the pairs.
- 1: actor, airport
"""

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=5000,
    eos_token_id=terminators,
    do_sample=True
    #temperature=0.6,
    #top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Here is the code box with the results in CSV format:

```
answers_csv
1: The actor, airport
2: The chef, attic
3: The coach, bank
4: The cowboy, barn
5: The dancer, castle
6: The detective, church
7: The doctor, clinic
8: The engineer, factory
9: The farmer, garage
10: The gardener, hallway
11: The hippie, hotel
12: The inventor, kitchen
13: The judge, laboratory
14: The maid, library
15: The musician, museum
16: The pilot, nightclub
17: The pirate, office
18: The queen, park
19: The scientist, prison
20: The sheriff, school
21: The soldier, stadium
22: The spy, studio
23: The teacher, temple
24: The tourist, theater
25: The accountant, beach
26: The artist, cinema
27: The dentist, hospital
28: The nurse, restaurant
29: The writer, station
30: The barista, mall
31: The firefighter, zoo
32: The lawyer, pharmacy
```
